# JSON Transformation

This data project has been used as a take-home assignment in the recruitment process for the data engineering positions at Chama.

Chama is a relatively new, modern company, with an IOS App and Android App. To grow our company we must make a special effort in collecting all information available around the App and make it available for everyone in the company. Some information about App usage is generated by our App and our backend API. Other sources of information, like the Google Play Store, can provide very useful insights on App performance and user rating. Chama is very data-oriented and, most decisions are made based on appropriate metrics, therefore, data quality is a must and a concern of everyone involved.

## Assignment

Some event data come as json files and need some transformation to be structured as tables. Convert the case.json file to 3 csv files, using the programming language of your choice, with the following rules:

1. `CuratedOfferOptions.csv`:

2. `DynamicPriceOption.csv`:

3. `DynamicPriceRange.csv:`

## General plan:
### 1. Load json file
### 2. Filter the relevant event type
If we examine the top-level json structure, 

In [65]:
{
    "EnqueuedTimeUtc": "2021-09-05 08:04:08 UTC",
    "EventName": "DynamicPrice_Result",
    "Payload": "{\"provider\":\"ApplyDynamicPriceRange\",\"offerId\":\"a6611d55-9624-4381-8cdd-323ee3689241\",\"algorithmOutput\":{\"min_global\":85.0,\"min_recommended\":87.2,\"max_recommended\":97.65,\"differenceMinRecommendMinTheory\":2.2}}"
  },

({'EnqueuedTimeUtc': '2021-09-05 08:04:08 UTC',
  'EventName': 'DynamicPrice_Result',
  'Payload': '{"provider":"ApplyDynamicPriceRange","offerId":"a6611d55-9624-4381-8cdd-323ee3689241","algorithmOutput":{"min_global":85.0,"min_recommended":87.2,"max_recommended":97.65,"differenceMinRecommendMinTheory":2.2}}'},)

Each record has an "EventName" that tells us what type of event it is. 
We use that to route the record to the correct CSV:

In [66]:
#if rec["EventName"] == "CurateOffer_Result":
#    # --> CurateOffer_Result.csv
#elif rec['EventName'] == "DynamicPrice_Result":
#    # --> DynamicPriceRange.csv or DynamicPriceOption.csv


### 3. Parse the Nested JSON Payload
The Payload field is a JSON-encoded string. You must decode it into a Python object (a list of dictionaries) using json.loads().
If decoding fails due to improper escaping, fallback to ast.literal_eval().

### 4. Iterate though nested structures
For each top_level record: iterate through each curation object in the decoded Payload. Then for each option inside each curation.

### 5. Format and collect the desired fields

- Extract each field according to the formatting rules 
- Use `dict.get("key","")` to avoid KeyErrors on missing fields
- Convert the UTC timestamp to Brazil timezone and reformat it as DD/MM/YYYY

### 6. Write to CSV files

## Python Code

In [67]:
# Need to validate the unique EventNames in this file
import json

# Load JSON
with open('/Users/rosiebai/Downloads/datasets-7/case.json', 'r') as f:
    data = json.load(f)

# Collect unique EventName values
event_names = set()

for rec in data:
    event = rec.get("EventName")
    event_names.add(event)

# Display the result
print(f"Unique EventNames ({len(event_names)}):")
for name in event_names:
    print("-", name)


Unique EventNames (2):
- CurateOffer_Result
- DynamicPrice_Result


In [68]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import ast

# Convert to Brazil time (UTC-3)
def convert_to_brazil_time(utc_str):
    dt = datetime.strptime(utc_str.replace(' UTC', ''), '%Y-%m-%d %H:%M:%S')
    dt = dt.replace(tzinfo=timezone.utc).astimezone(timezone(timedelta(hours=-3)))
    return dt.strftime('%d/%m/%Y')

# Extract rows
curated_rows = []

for rec in data:
    if rec.get("EventName") != "CurateOffer_Result":
        continue

    enqueued_time_br = convert_to_brazil_time(rec["EnqueuedTimeUtc"])
    payload = rec.get("Payload")

    try:
        parsed_payload = json.loads(payload)
    except json.JSONDecodeError:
        parsed_payload = ast.literal_eval(payload)

    for curation in parsed_payload:
        provider = curation.get("curationProvider", "")
        offer_id = curation.get("offerId", "")
        dealer_id = curation.get("dealerId", "")
        for option in curation.get("options", []):
            row = {
                "CurationProvider": f'"{provider}"',
                "OfferId": f'"{offer_id}"',
                "DealerId": f'"{dealer_id}"',
                "UniqueOptionId": f'"{option.get("uniqueOptionId", "")}"',
                "OptionId": f'"{option.get("optionId", "")}"',
                "IsMobileDealer": option.get("isMobileDealer", ""),
                "IsOpen": option.get("isOpen", ""),
                "Eta": f'"{option.get("eta", "")}"',
                "ChamaScore": option.get("chamaScore", ""),
                "ProductBrand": f'"{option.get("productBrand", "")}"',
                "IsWinner": option.get("isWinner", ""),
                "MinimumPrice": option.get("minimumPrice", ""),
                "MaximumPrice": option.get("maximumPrice", ""),
                "DynamicPrice": option.get("dynamicPrice", ""),
                "FinalPrice": option.get("finalPrice", ""),
                "DefeatPrimaryReason": f'"{option.get("defeatPrimaryReason", "")}"',
                "DefeatReasons": f'"{",".join(option.get("defeatReasons", []))}"' if option.get("defeatReasons") else '""',
                "EnqueuedTimeSP": enqueued_time_br
            }
            curated_rows.append(row)

# Export to CSV
CuratedOfferOptions = pd.DataFrame(curated_rows)
CuratedOfferOptions.to_csv("/Users/rosiebai/Desktop/python practice/CuratedOfferOptions.csv", index=False)


In [69]:
CuratedOfferOptions.head()

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/2021
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""Closed""","""Closed,HasDriverInOffer""",25/08/2021
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.00,198.00,99.95,99.95,"""Closed""","""Closed,HigherPrice,HasDriverInOffer""",25/08/2021
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""Closed,HigherPrice,HigherETA""",25/08/2021
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.00,184.00,93.90,93.90,"""Closed""","""Closed,HigherPrice,HigherETA""",25/08/2021


In [70]:
# Collect rows for CSV
option_rows = []

for rec in data:
    if rec.get("EventName") != "DynamicPrice_Result":
        continue

    enqueued_time_br = convert_to_brazil_time(rec["EnqueuedTimeUtc"])
    payload = rec.get("Payload")

    try:
        parsed_payload = json.loads(payload)
    except json.JSONDecodeError:
        parsed_payload = ast.literal_eval(payload)

    # Only handle option-based pricing here (list of options)
    algo_output = parsed_payload.get("algorithmOutput")
    if not isinstance(algo_output, list):
        continue

    provider = parsed_payload.get("provider", "")
    offer_id = parsed_payload.get("offerId", "")

    for option in algo_output:
        option_rows.append({
            "Provider": f'"{provider}"',
            "OfferId": f'"{offer_id}"',
            "UniqueOptionId": f'"{option.get("uniqueOptionId", "")}"',
            "BestPrice": option.get("bestPrice", ""),
            "EnqueuedTimeSP": enqueued_time_br
        })

# Save as CSV
DynamicPriceOption = pd.DataFrame(option_rows)
DynamicPriceOption.to_csv("/Users/rosiebai/Desktop/python practice/DynamicPriceOption.csv", index=False)

In [71]:
DynamicPriceOption

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/2021
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/2021
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/2021
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/2021
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/2021
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/2021
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/2021
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/2021
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/2021
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/2021


In [72]:
# Collect rows for CSV
range_rows = []
for rec in data:
    if rec.get("EventName") != "DynamicPrice_Result":
        continue 
    enqueued_time_br = convert_to_brazil_time(rec["EnqueuedTimeUtc"])
    payload = rec.get("Payload")
    
    try:
        parsed_payload = json.loads(payload)
    except json.JSONDecodeError:
        parsed_payload = ast.literal_eval(payload)
    
    algo_output = parsed_payload.get("algorithmOutput")
    if not isinstance(algo_output, dict):
        continue 

provider = parsed_payload.get("provider","")
offer_id = parsed_payload.get("offerId","")
for option in algo_output:
    range_rows.append({
        "Provider":f'"{provider}"',
        "OfferId":f'"{offer_id}"',
        "MinGlobal":algo_output.get("min_global",""),
        "MinRecommended": algo_output.get("min_recommended",""),
        "MaxRecommended": algo_output.get("max_recommended",""),
        "DifferenceMinRecommendMinTheory": algo_output.get("differenceMinRecommendMinTheory",""),
        "EnqueuedTimeSP": enqueued_time_br
        
    })
# Save as CSV 

DynamicPriceRange = pd.DataFrame(range_rows)
DynamicPriceRange.to_csv("/Users/rosiebai/Desktop/python practice/DynamicPriceRange.csv", index=False)


In [73]:
DynamicPriceRange

,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""92269c83-9103-4ea3-b93b-ae2f1762f720""",85.0,87.2,97.65,2.2,05/09/2021
1,"""ApplyDynamicPriceRange""","""92269c83-9103-4ea3-b93b-ae2f1762f720""",85.0,87.2,97.65,2.2,05/09/2021
2,"""ApplyDynamicPriceRange""","""92269c83-9103-4ea3-b93b-ae2f1762f720""",85.0,87.2,97.65,2.2,05/09/2021
3,"""ApplyDynamicPriceRange""","""92269c83-9103-4ea3-b93b-ae2f1762f720""",85.0,87.2,97.65,2.2,05/09/2021


## Validation

Step 1: Count JSON records 

In [74]:
with open('/Users/rosiebai/Downloads/datasets-7/case.json', 'r') as f:
    records = json.load(f)
print(f"Total top-level records in JSON: {len(records)}")

Total top-level records in JSON: 37


This gives us the total umber of JSON entries.
We can then summarize by EventName:

In [75]:
from collections import Counter 
event_counts = Counter(rec.get("EventName") for rec in records)
print("Event counts:", event_counts)

Event counts: Counter({'DynamicPrice_Result': 32, 'CurateOffer_Result': 5})


Step 2: Track how many rows each csv file will have 

In [76]:
# After processing:
print(f"CuratedOfferOptions.csv --> {len(curated_rows)} rows")
print(f"DynamicPriceOption.csv --> {len(option_rows)} rows")
print(f"DynamicPriceRange.csv --> {len(range_rows)} rows")


CuratedOfferOptions.csv --> 40 rows
DynamicPriceOption.csv --> 56 rows
DynamicPriceRange.csv --> 4 rows


Step 3: Post-write file validation 

In [77]:
import pandas 
df1 = pd.read_csv("CuratedOfferOptions.csv")
df2 = pd.read_csv("DynamicPriceOption.csv")
df3 = pd.read_csv("DynamicPriceRange.csv")

print("Rows in saved CuratedOfferOptions.csv:", len(df1))
print("Rows in saved DynamicPriceOption.csv", len(df2))
print("Rows in saved DynamicPriceRange.csv", len(df3))

Rows in saved CuratedOfferOptions.csv: 40
Rows in saved DynamicPriceOption.csv 56
Rows in saved DynamicPriceRange.csv 4
